In [1]:
import numpy as np
import threading
import pickle
import os
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
MODEL_VERSION = 8

## 数据预处理

In [2]:
# 训练集
# pd_train_data = pd.read_csv(r"adult/adult.data", header=None)
# pd_train_data_missing_to_nan = pd_train_data.replace("\?", np.nan, regex=True)
# pd_train_data_missing_to_mode = pd_train_data_missing_to_nan.fillna(value=pd_train_data_missing_to_nan.mode().iloc[0])
# X_train = pd_train_data_missing_to_mode.iloc[:, :14].values
# y_train = pd_train_data_missing_to_mode.iloc[:, 14].values
data = np.genfromtxt("adult/adult.data", delimiter=",", dtype=str)
data = data[~np.any(data == '?', axis=1)]
X_train = data[:, :14]
y_train = data[:, 14]


# 测试集
# pd_test_data = pd.read_csv(r"adult/adult.test", header=None, skiprows=1)
# pd_test_data_missing_to_nan = pd_test_data.replace("\?", np.nan, regex=True)
# pd_test_data_missing_to_mode = pd_test_data_missing_to_nan.fillna(value=pd_test_data_missing_to_nan.mode().iloc[0])
# X_test = pd_test_data_missing_to_mode.iloc[:, :14].values
# y_test = pd_test_data_missing_to_mode.iloc[:, 14].values
data_test = np.genfromtxt("adult/adult.test", delimiter=",", dtype=str, skip_header=1)
data_test = data_test[~np.any(data_test == '?', axis=1)]
X_test = data_test[:, :14]
y_test = data_test[:, 14]


# 01编码
true_sample = y_train[0]
y_train = y_train == np.full(y_train.shape[0], true_sample)
y_test = y_test == np.full(y_test.shape[0], true_sample + '.')

# 数值编码
for i in [1, 3, 5, 6, 7, 8, 9, 13]:
    values_train = np.unique(X_train[:, i])
    values_test = np.unique(X_test[:, i])
    values = np.append(values_train, values_test)
    values = np.unique(values)
    encoding = {value: idx for idx, value in enumerate(values)}
    for j in range(X_train.shape[0]):
        X_train[j][i] = encoding[X_train[j][i]]
    for j in range(X_test.shape[0]):
        X_test[j][i] = encoding[X_test[j][i]]

X_train = X_train.astype(int)
y_train = y_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
X_train
y_train

array([[    39,      7,  77516, ...,      0,     40,     39],
       [    50,      6,  83311, ...,      0,     13,     39],
       [    38,      4, 215646, ...,      0,     40,     39],
       ...,
       [    58,      4, 151910, ...,      0,     40,     39],
       [    22,      4, 201490, ...,      0,     20,     39],
       [    52,      5, 287927, ...,      0,     40,     39]])

array([1, 1, 1, ..., 1, 1, 0])

## 得分

In [3]:
class score(object):
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
    
    @property
    def accuracy_score(self):
        correct = np.sum(self.y_true == self.y_pred)
        total = len(self.y_true)
        accuracy = correct / total
        return accuracy
    
    @property
    def precision_score(self):
        true_positive = np.sum((self.y_true == 1) & (self.y_pred == 1))
        false_positive = np.sum((self.y_true == 0) & (self.y_pred == 1))
        precision = true_positive / (true_positive + false_positive)
        return precision
    
    @property
    def recall_score(self):
        true_positive = np.sum((self.y_true == 1) & (self.y_pred == 1))
        false_negative = np.sum((self.y_true == 1) & (self.y_pred == 0))
        recall = true_positive / (true_positive + false_negative)
        return recall
    
    @property
    def f1_score(self):
        precision = self.precision_score
        recall = self.recall_score
        f1 = 2 * precision * recall / (precision + recall)
        return f1


## 数据划分（基于信息熵与信息增益）

In [4]:
def calculate_entropy(y):
    """信息熵"""
    classes = np.unique(y)
    entropy = 0
    for c in classes:
        p = np.sum(y == c) / len(y)
        entropy -= p * np.log2(p)
    return entropy

def calculate_information_gain(X, y, feature_index, threshold):
    """信息增益"""
    parent_entropy = calculate_entropy(y)
    
    left_indices = X[:, feature_index] <= threshold
    right_indices = ~left_indices
    
    left_entropy = calculate_entropy(y[left_indices])
    right_entropy = calculate_entropy(y[right_indices])
    
    n_total = len(y)
    n_left = np.sum(left_indices)
    n_right = n_total - n_left
    
    child_entropy = (n_left / n_total) * left_entropy + (n_right / n_total) * right_entropy
    
    information_gain = parent_entropy - child_entropy
    return information_gain

def find_best_split(X, y):
    """数据划分"""
    best_feature_index = None
    best_threshold = None
    best_information_gain_ratio = -np.inf
    parent_entropy = calculate_entropy(y)
    
    for feature_index in range(X.shape[1]):
        values = X[:, feature_index]
        unique_values = np.unique(values)
        if len(unique_values) <= 1:
            continue
        for value in unique_values:
            left_indices = values <= value
            right_indices = ~left_indices
            left_entropy = calculate_entropy(y[left_indices])
            right_entropy = calculate_entropy(y[right_indices])
            conditional_entropy = ((np.sum(left_indices) / len(y)) * left_entropy 
                                   + (np.sum(right_indices) / len(y)) * right_entropy)
            information_gain = parent_entropy - conditional_entropy
            split_info = calculate_entropy(values)
            information_gain_ratio = information_gain / split_info
            if information_gain_ratio > best_information_gain_ratio:
                best_information_gain_ratio = information_gain_ratio
                best_feature_index = feature_index
                best_threshold = value
    
    return best_feature_index, best_threshold

## 结点

In [5]:
class TreeNode:
    def __init__(
            self, feature_index=None, threshold=None, 
            value=None, left=None, right=None
    ):
        self.feature_index = feature_index
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

## 决策树

In [6]:
class DecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None
    
    def generateTree(self, name, parent: TreeNode=None, depth=1):
        X, y = self.load_Xy(name)
        self.del_file(name)
        
        if depth == self.max_depth or len(set(y)) == 1:
            node = TreeNode(value=np.sort(y)[y.shape[0] >> 1])
        else:
            best_feature_index, best_threshold = find_best_split(X, y)
            left_indices = X[:, best_feature_index] <= best_threshold
            right_indices = ~left_indices
            
            self.save_Xy(X[left_indices], y[left_indices], name + '0')
            self.save_Xy(X[right_indices], y[right_indices], name + '1')
            
            node = TreeNode(
                feature_index=best_feature_index, threshold=best_threshold,
                left=None, right=None
            )
            
            thread1 = threading.Thread(
                target=self.generateTree, args=(name + '0', node, depth + 1,)
            )
            thread2 = threading.Thread(
                target=self.generateTree, args=(name + '1', node, depth + 1,)
            )
            
            thread1.start()
            thread2.start()
            
            thread1.join()
            thread2.join()
        
        if name[-1] == '0':
            parent.left = node
        elif name[-1] == '1':
            parent.right = node
        else:
            self.root = node
        return None
    
    def fit(self, X, y, name='X', save=False):
        print('training')
        self.save_Xy(X, y, name)
        self.generateTree(name)
        if save:
            self.saveTree(f'Models/Tree_Model_{MODEL_VERSION}.pkl')
        print('ok')
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """预测逻辑"""
        y_pred = np.array([None] * X.shape[0])
        for i in range(X.shape[0]):
            node = self.root
            while node.left is not None and node.right is not None:
                if X[i][node.feature_index] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            y_pred[i] = node.value
        return y_pred
    
    @staticmethod
    def load_Xy(filename):
        X = np.load(f'temp/{filename}_X.npy', allow_pickle=True)
        y = np.load(f'temp/{filename}_y.npy', allow_pickle=True)
        return X, y
    
    @staticmethod
    def save_Xy(X, y, filename):
        np.save(f'temp/{filename}_X', X)
        np.save(f'temp/{filename}_y', y)
    
    @staticmethod
    def del_file(filename):
        os.remove(f'temp/{filename}_X.npy')
        os.remove(f'temp/{filename}_y.npy')

    def saveTree(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.root, f)
    
    def loadTree(self, filename):
        with open(filename, 'rb') as f:
            self.root = pickle.load(f)

## 决策森林

In [7]:
class DecisionForest(object):
    def __init__(self):
        self.tree: list[DecisionTree|None] = []
        self.weight: list[float|None] = []
        self.tree_number = None
    
    def fit(self, X: np.ndarray, y: np.ndarray, tree_number: int=5, *args, **kwargs):
        print('Forest training start')
        
        self.tree_number =  tree_number
        every_sample_number = X.shape[0] // tree_number
        
        # 训练
        threads = []
        for i in range(tree_number):
            self.tree.append(DecisionTree(*args, **kwargs))
            threads.append(
                threading.Thread(
                    target=self.tree[i].fit, args=(
                        X[every_sample_number * i: every_sample_number * (i + 1)],
                        y[every_sample_number * i: every_sample_number * (i + 1)],
                        self.transName(i),
                    )
                )
            )
            threads[i].start()
        
        for i in range(tree_number):
            threads[i].join()
        
        # 加权
        pred = np.array([sum(tree.predict(X) == y) for tree in self.tree])
        self.weight = np.array([p / sum(pred) for p in pred])
        
        self.saveForest(f'Models/Forest_model_{MODEL_VERSION}.pkl')
        
        print('Forest is ok')
    
    def predict(self, X):
        votes = np.zeros((self.tree_number, X.shape[0]))
        for i in range(self.tree_number):
            votes[i] = self.tree[i].predict(X)
        votes = np.array([np.dot(vote, self.weight) >= 0.5 for vote in votes.T])
        return votes
    
    def saveForest(self, filename):
        with open(filename + '-tree', 'wb') as f:
            pickle.dump(self.tree, f)
        with open(filename + '-weight', 'wb') as f:
            pickle.dump(self.weight, f)
    
    def loadForest(self, filename):
        with open(filename + '-tree.npy', 'rb') as f:
            self.tree = pickle.load(f)
        with open(filename + 'weight.npy', 'rb') as f:
            self.weight = pickle.load(f)
    
    @staticmethod    
    def transName(k: int) -> str:
        return 'a' * (k // 26) + chr(ord('a') + k % 26)
        

## 模型训练

In [8]:
forest = DecisionForest()
forest.fit(X_train, y_train, tree_number=100, max_depth=5)

Forest training start
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
oko

## 模型测试

In [9]:
y_pred_forest = forest.predict(X_test)
sc = score(y_test, y_pred_forest)
print("准确率: ", sc.accuracy_score, '\n',
      "精确率: ", sc.precision_score, '\n',
      "召回率: ", sc.recall_score, '\n', 
      "F1 值: ", sc.f1_score, '\n')

准确率:  0.8201584669246361 
 精确率:  0.8138245197068726 
 召回率:  0.9913148371531966 
 F1 值:  0.8938438111812053 


In [10]:
# sklearn测试
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_forest)
print("准确率:", accuracy)

precision = precision_score(y_test, y_pred_forest, pos_label=1)
print("精确率:", precision)

recall = recall_score(y_test, y_pred_forest, pos_label=1)
print("召回率:", recall)

f1 = f1_score(y_test, y_pred_forest, pos_label=1)
print("F1 值:", f1)


准确率: 0.8201584669246361
精确率: 0.8138245197068726
召回率: 0.9913148371531966
F1 值: 0.8938438111812053


## 测试代码

In [11]:
    # def draw_tree(self, root, x=0, y=0, width=50, height=10):
    #     if root is None:
    #         return
    #     # 绘制当前节点
    #     plt.text(
    #         x, y, str(root), color='black', fontsize=12, ha='center', va='center', 
    #         bbox=dict(facecolor='white', edgecolor='black', boxstyle='circle')
    #     )
    #     # 绘制左子树
    #     if root.left:
    #         plt.plot([x, x - width], [y, y - height], color='black')
    #         self.draw_tree(root.left, x - width, y - height, width / 2, height)
    #     # 绘制右子树
    #     if root.right:
    #         plt.plot([x, x + width], [y, y - height], color='black')
    #         self.draw_tree(root.right, x + width, y - height, width / 2, height)
    # 
    # def show_tree(self):
    #     plt.figure(figsize=(8, 6))
    #     self.draw_tree(self.root)
    #     plt.axis('off')
    #     plt.show()

In [49]:
import numpy as np
import pandas as pd
from IPython.display import display
from fractions import Fraction
def distance(x1, x2):
    return np.sum(np.square(x1 - x2))


def distance_matrix(Means: np.ndarray, points: np.ndarray):
    Matrix = pd.DataFrame(columns=np.append(Means[:, -1], 'Nearest'), index=points[:, -1])

    for X in points:
        for Y in Means:
            x = X[:-1].astype(float)
            y = Y[:-1].astype(float)
            Matrix.loc[X[-1], Y[-1]] = distance(np.array(x), np.array(y))
        Matrix.loc[X[-1], 'Nearest'] = Matrix.columns[np.argmin(Matrix.loc[X[-1]])]
        
    new_Means = np.array(
        [points[Matrix['Nearest'] == it][:, :-1].astype(float).mean(axis=0).tolist() + [it]
        for it in ['A', 'B', 'C']]
    )
    return Matrix, new_Means


def main():
    points = np.array([
        [2, 10, "A1"],  # A1
        [2, 5, "A2"],  # A2
        [8, 4, "A3"],  # A3
        [5, 8, "B1"],  # B1
        [7, 5, "B2"],  # B2
        [6, 4, "B3"],  # B3
        [1, 2, "C1"],  # C1
        [4, 9, "C3"]  # C3
    ])
    
    Means = np.array([
        [2, 10, "A"],  # A
        [5, 8, "B"],  # B
        [1, 2, "C"]  # C
    ])
    

    for i in range(5):
        print(f'iteration {i+1}')
        # display(pd.DataFrame(
        #     points[:, :-1],
        #     columns=['x', 'y'], 
        #     index=points[:, -1])
        # )
        display(pd.DataFrame(Means[:, :-1], columns=['x', 'y'], index=Means[:, -1]))
        Matrix, new_Means = distance_matrix(Means, points)
        Matrix = Matrix.applymap(lambda x: x * 18 if isinstance(x, float) else x)
        display(Matrix)
        Means = new_Means
    
    
main()
# np.sum(np.square(np.array([1, 2]) - np.array([2, 3])))

iteration 1


,x,y
A,2,10
B,5,8
C,1,2


,A,B,C,Nearest
A1,0.0,234.0,1170.0,A
A2,450.0,324.0,180.0,C
A3,1296.0,450.0,954.0,B
B1,234.0,0.0,936.0,B
B2,900.0,234.0,810.0,B
B3,936.0,306.0,522.0,B
C1,1170.0,936.0,0.0,C
C3,90.0,36.0,1044.0,B


iteration 2


,x,y
A,2.0,10.0
B,6.0,6.0
C,1.5,3.5


...

,x,y
A,3.6666666666666665,9.0
B,7.0,4.333333333333333
C,1.5,3.5


,A,B,C,Nearest
A1,68.0,1028.0,765.0,A
A2,338.0,458.0,45.0,C
A3,788.0,20.0,765.0,B
B1,50.0,314.0,585.0,A
B2,488.0,8.0,585.0,B
B3,548.0,20.0,369.0,B
C1,1010.0,746.0,45.0,C
C3,2.0,554.0,657.0,A


iteration 5


,x,y
A,3.6666666666666665,9.0
B,7.0,4.333333333333333
C,1.5,3.5


,A,B,C,Nearest
A1,68.0,1028.0,765.0,A
A2,338.0,458.0,45.0,C
A3,788.0,20.0,765.0,B
B1,50.0,314.0,585.0,A
B2,488.0,8.0,585.0,B
B3,548.0,20.0,369.0,B
C1,1010.0,746.0,45.0,C
C3,2.0,554.0,657.0,A
